# Load env and files

In [1]:
import os
os.chdir('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from fossil_classification import *
from enrich_holdings import *
from reports_etl import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.2f}".format

In [4]:
response_path = "data/downloaded reports/company reports 2021Q3/"
all_holdings_cls_path = response_path + "all_holdings_cls.csv"
all_holdings_cls = pd.read_csv(all_holdings_cls_path, dtype=holdings_dtypes())

tlv_s2i = prepare_tlv_sec_num_to_issuer(fetch_latest_tlv_sec_num_to_issuer())
isin2lei = fetch_latest_isin2lei()

/Users/urimarom/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,7,13,14,20,21,22,23,24,25,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# adding "I_" to ParentCorpLegalId to avoid confusion with il_corp_num
all_holdings_cls["ParentCorpLegalId"] = "I_" + all_holdings_cls["ParentCorpLegalId"]

In [6]:
# enrich holdings file - fix IDs
all_holdings_cls = add_all_id_types_to_holdings(all_holdings_cls, tlv_s2i, isin2lei)

/Users/urimarom/anaconda3/lib/python3.6/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)



Holding file ISIN col is: מספר ני"ע
number of ISINs: 88110 out of 557933 rows

Holding file מספר תאגיד col is: מספר מנפיק
number of מספר תאגידs: 185920 out of 557933 rows
מספר ני"עs with matching ISIN: 194797 out of total relevant rows: 467615
מספר תאגידs with matching מספר מנפיק: 207991 out of total relevant rows: 185920
מספר ני"עs with matching מספר מנפיק: 209864 out of total relevant rows: 467615
ISINs with matching מספר מנפיק: 210320 out of total relevant rows: 194797
ISINs with matching LEI: 47113 out of total relevant rows: 194797


# Let the Analysis begin!

In [7]:
pd.DataFrame(all_holdings_cls.groupby(['SystemName'])['שווי'].agg('sum').map('{:,.2f}'.format))

,שווי
SystemName,
ביטוח,"2,435,232,470.44"
גמל,"3,788,403,824.16"
פנסיה,"3,491,417,540.04"


In [8]:
def get_summary(arg1, *args, holdings=all_holdings_cls):
    '''get summary stats grouped by 1 or more columns, e.g. Company, holding_type
    '''
    group = [arg1]+[*args]
    summary = pd.DataFrame(holdings.groupby(group, dropna=False).agg(
    {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    summary["שיעור פוסילי מסך הנכסים"] = 100 * summary["שווי פוסילי"] / summary["שווי"]
    # add summary of stocks and bonds only
    holdings_cls_trd_stocks_bonds = holdings[holdings["holding_type"].isin(['מניות', 'אג"ח קונצרני'])]
    summary_tradable_stocks_bonds_only = pd.DataFrame(holdings_cls_trd_stocks_bonds.groupby(group, dropna=False).agg(
    {'שווי':'sum', 'שווי פוסילי':'sum'})).reset_index()
    summary_tradable_stocks_bonds_only["שיעור פוסילי במניות ואגח קונצרני סחירים"] = 100 * summary_tradable_stocks_bonds_only[
        "שווי פוסילי"] / summary_tradable_stocks_bonds_only["שווי"]
    summary = summary.merge(summary_tradable_stocks_bonds_only,
                            left_on=group,
                            right_on=group,
                            how='left',
                            suffixes = ['', ' במניות ואגח קונצרני סחירים']
                           )
    return summary

## Get Company level stats over time

### Filter results to include only major companies

In [9]:
institutions = [
    'מנורה',
    'מגדל',
    'הראל',
    'מיטב דש',
    'אלטשולר',
    'מור',
    'הפניקס',
    'כלל',
    'פסגות',
    'הלמן',
    'אנליסט',
    'ילין'
]

mask = all_holdings_cls["ParentCorpName"].str.startswith(tuple(institutions))

# removing הלמן-אלדובי חח"י גמל בע"מ 515447035 & 520027715, מנורה מבטחים והסתדרות המהנדסים ניהול קופות גמל בע"מ
all_holdings_cls_filtered = all_holdings_cls[mask][~all_holdings_cls[mask]["ParentCorpLegalId"].isin(['520027715', '515447035'])]
# all_holdings_cls_filtered.groupby(["ParentCorpName","ParentCorpLegalId"]).size()

all_holdings_cls_filtered['ParentCorpGroup'] = all_holdings_cls_filtered['ParentCorpName'].str.split().str[0]
all_holdings_cls_filtered['ReportPeriodDate'] = all_holdings_cls_filtered['ReportPeriodDesc'].map(report_period_desc_to_date)
company_stats_since_2020 = get_summary('ReportPeriodDate', 'ParentCorpGroup', holdings=all_holdings_cls_filtered)

In [10]:
company_stats_since_2020.to_csv(response_path+"company_stats_since_2020.csv", index=False)

## Export holdings with missing classification that should be classified

In [12]:
missing_cls = all_holdings_cls[
    (all_holdings_cls["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls["is_fossil"].isnull())
]

cols = ['שם המנפיק/שם נייר ערך' ,'מספר ני"ע' ,'מספר מנפיק', 'ISIN', 'מספר תאגיד', 'LEI']
missing_cls_deduped = missing_cls[cols].drop_duplicates()
print(missing_cls_deduped.shape[0])
missing_cls_deduped

13


,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,ISIN,מספר תאגיד,LEI
33299,CHKP US Equity- צ'ק פוינט,IL0010824113 EQUITY,NaN,None,520042821,NaN
43826,PLURISTEM Therapeutics Inc SYS- PLURISTEM THER...,US72940P1066-71013122,10337,None,None,NaN
60336,PEMEX 5.95 28/01/31,USP71654QDE98,4768,None,None,NaN
65698,GLAXOSMITHKLINE PLC- GLAXOSMITHKLINE PLC,US37733W1053-110637592,10172,None,None,NaN
114513,Tencent holdings- Tencent holdings ltd,KY6875721220-60175411,11074,None,None,NaN
137899,איי.אפ.אפ- אינטרנשיונל פליוורס אנד פראגרנסס אינק,US4595061015_1155019,1760,None,None,NaN
146015,Capital One- capital one,US14040H1059-60004140,11253,None,None,NaN
168984,P-ADR,70422225,NaN,None,None,NaN
218873,LVS VEGAS SANDS CORP- LAS VEGAS SANDS CORP,71032965 - US5178341070,28853,None,None,NaN
346953,CYBR 0 11/15/2024,US23248VAA35 C,NaN,None,None,NaN


In [15]:
missing_cls_deduped.to_csv(response_path+"missing_cls.csv", index=False)

## Compare Fossil Holdings over quarters

In [13]:
quarters_for_comparison = ['2021 רבעון 2', '2021 רבעון 3']
company_for_comparison = ['אלטשולר']

majors_fossil_holdings_for_comparison = all_holdings_cls_filtered[
    (all_holdings_cls_filtered["holding_type"].isin(['מניות', 'אג"ח קונצרני'])) &
    (all_holdings_cls_filtered["is_fossil"] == 1) &
    (all_holdings_cls_filtered["ReportPeriodDesc"].isin(quarters_for_comparison)) &
    (all_holdings_cls_filtered["ParentCorpGroup"].isin(company_for_comparison))
]

majors_fossil_holdings_for_comparison["שווי פוסילי"].sum()

5934869.925977995

In [15]:
majors_fossil_holdings_for_comparison[
    (majors_fossil_holdings_for_comparison["מספר מנפיק"].isnull()) &
    (majors_fossil_holdings_for_comparison["LEI"].isnull())
].head()

,שם המנפיק/שם נייר ערך,"מספר ני""ע",מספר מנפיק,דירוג,שם מדרג,סוג מטבע,שעור ריבית,תשואה לפדיון,שווי,שעור מנכסי אפיק ההשקעה,שעור מסך נכסי השקעה,report_id,holding_type,זירת מסחר,תאריך רכישה,"מח""מ",ערך נקוב,שער,פדיון/ריבית/דיבידנד לקבל,שעור מערך נקוב מונפק,ספק מידע,ענף מסחר,נכס בסיס,קונסורציום כן/לא,תאריך שערוך אחרון,אופי הנכס,שעור תשואה במהלך התקופה,כתובת הנכס,ריבית אפקטיבית,SystemName,ParentCorpName,ParentCorpLegalId,ProductNum,Name,ShortName,StatusDate,ReportPeriodDesc,is_fossil,שווי פוסילי,ISIN,מספר תאגיד,LEI,ParentCorpGroup,ReportPeriodDate
343647,PEMEX 4 3/4 02/26/29,None,NaN,BB,S&P,אירו,0.05,0.05,"6,202.05",0.01,0.00,2105696,"אג""ח קונצרני",אחר,NaN,6.18,"1,584,000.00",101.05,0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"אלטשולר שחם גמל ופנסיה בע""מ",I_513173393,0.0,nan,nan,06/09/2021,2021 רבעון 2,1.00,"6,202.05",XS1824424706,None,NaN,אלטשולר,2021-06-30
343651,ORO NEGRO DRILLING,None,NaN,NR,NaN,דולר אמריקאי,0.07,0.00,0.01,0.00,0.00,2105696,"אג""ח קונצרני",אחר,NaN,0.00,"4,059.00",0.06,0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"אלטשולר שחם גמל ופנסיה בע""מ",I_513173393,0.0,nan,nan,06/09/2021,2021 רבעון 2,1.00,0.01,NO0010838550,None,NaN,אלטשולר,2021-06-30
343652,ORO NEGRO DRILLING,None,NaN,NR,NaN,דולר אמריקאי,0.07,0.00,0.01,0.00,0.00,2105696,"אג""ח קונצרני",אחר,NaN,0.72,"4,059.00",0.06,0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"אלטשולר שחם גמל ופנסיה בע""מ",I_513173393,0.0,nan,nan,06/09/2021,2021 רבעון 2,1.00,0.01,NO0010843022,None,NaN,אלטשולר,2021-06-30
343653,ORO NEGRO DRILLING,None,NaN,NR,NaN,דולר אמריקאי,0.07,0.00,0.01,0.00,0.00,2105696,"אג""ח קונצרני",אחר,NaN,0.00,"4,059.00",0.06,0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"אלטשולר שחם גמל ופנסיה בע""מ",I_513173393,0.0,nan,nan,06/09/2021,2021 רבעון 2,1.00,0.01,NO0010838592,None,NaN,אלטשולר,2021-06-30
343654,ORO NEGRO DRILLING,None,NaN,NR,NaN,דולר אמריקאי,0.07,0.00,0.01,0.00,0.00,2105696,"אג""ח קונצרני",אחר,NaN,0.00,"4,059.00",0.06,0.00,0.00,בלומברג,Energy,NaN,NaN,NaN,NaN,nan,NaN,nan,פנסיה,"אלטשולר שחם גמל ופנסיה בע""מ",I_513173393,0.0,nan,nan,06/09/2021,2021 רבעון 2,1.00,0.01,NO0010838634,None,NaN,אלטשולר,2021-06-30


## group holdings for summary stats per held company

### Handle Israeli holdings (having Israeli security num)

In [16]:
# 1. group by Israeli security number
il_fossil_holdings = majors_fossil_holdings_for_comparison.groupby(["ParentCorpGroup", "ReportPeriodDate", 'מספר ני"ע'])
il_fossil_holdings_agg = il_fossil_holdings.agg(
    name=pd.NamedAgg(column="שם המנפיק/שם נייר ערך", aggfunc="first"),
    issuer_num=pd.NamedAgg(column="מספר מנפיק", aggfunc="first"),
    il_corp_num=pd.NamedAgg(column="מספר תאגיד", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="שווי פוסילי", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="ערך נקוב", aggfunc="sum")
).reset_index()

In [17]:
# use il_corp_num when issuer_num when missing
il_fossil_holdings_agg["issuer_num"] = il_fossil_holdings_agg["issuer_num"].fillna(il_fossil_holdings_agg["il_corp_num"])

In [18]:
# 2. group by Israeli issuer number
il_fossil_holdings_by_issuer = il_fossil_holdings_agg.groupby(["ParentCorpGroup", "ReportPeriodDate", 'issuer_num'])
il_fossil_holdings_by_issuer_agg = il_fossil_holdings_by_issuer.agg(
    name=pd.NamedAgg(column="name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

In [19]:
il_fossil_holdings_by_issuer_agg["fossil_sum"].sum()

4106236.092732214

In [20]:
majors_fossil_holdings_for_comparison[majors_fossil_holdings_for_comparison['מספר ני"ע'].notnull()]["שווי פוסילי"].sum()

4106236.092732214

Great Success!

### Handle non-Israeli holdings

In [21]:
non_il_fossil_holdings = majors_fossil_holdings_for_comparison[majors_fossil_holdings_for_comparison['מספר ני"ע'].isnull()]
non_il_fossil_holdings["שווי פוסילי"].sum()

1828633.833245781

In [22]:
non_il_fossil_holdings["ISIN"].isnull().sum()

0

In [23]:
# 1. group by ISIN
non_il_fossil_holdings = non_il_fossil_holdings.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'ISIN'
])
non_il_fossil_holdings_agg = non_il_fossil_holdings.agg(
    name=pd.NamedAgg(column="שם המנפיק/שם נייר ערך", aggfunc="first"),
    issuer_num=pd.NamedAgg(column="מספר מנפיק", aggfunc="first"),
    il_corp_num=pd.NamedAgg(column="מספר תאגיד", aggfunc="first"),
    lei=pd.NamedAgg(column="LEI", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="שווי פוסילי", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="ערך נקוב", aggfunc="sum")
).reset_index()

print(non_il_fossil_holdings_agg["fossil_sum"].sum())
# Fill in issuer_num for missing LEIs, then il_corp_num
non_il_fossil_holdings_agg["issuer_num"] = non_il_fossil_holdings_agg["issuer_num"].fillna(
    non_il_fossil_holdings_agg["lei"])
non_il_fossil_holdings_agg["issuer_num"] = non_il_fossil_holdings_agg["issuer_num"].fillna(
    non_il_fossil_holdings_agg["il_corp_num"])
non_il_fossil_holdings_agg.head()

1828633.833245781


,ParentCorpGroup,ReportPeriodDate,ISIN,name,issuer_num,il_corp_num,lei,fossil_sum,quantity_sum
0,אלטשולר,2021-06-30,IL0028102734,"Icl 4.5% 02/12/2024- איי.סי.אל גרופ בע""מ (דואלי)",281,520027830,None,"98,288.49","27,500,000.00"
1,אלטשולר,2021-06-30,NO0010700982,Oro negro dril 7.5% 2019- Oro negro dril pte ltd,12824,None,None,"1,843.93","10,284,019.01"
2,אלטשולר,2021-06-30,NO0010838550,Oro Negro Drilling- Oro negro dril pte ltd,12824,None,None,0.72,"400,462.00"
3,אלטשולר,2021-06-30,NO0010838584,Oro Negro Drilling- Oro negro dril pte ltd,12824,None,None,0.72,"400,462.00"
4,אלטשולר,2021-06-30,NO0010838592,Oro Negro Drilling- Oro negro dril pte ltd,12824,None,None,0.72,"400,462.00"


In [24]:
non_il_fossil_holdings_by_issuer = non_il_fossil_holdings_agg.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'issuer_num'
], dropna=False)
non_il_fossil_holdings_by_issuer_agg = non_il_fossil_holdings_by_issuer.agg(
    name=pd.NamedAgg(column="name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

In [25]:
non_il_fossil_holdings_by_issuer_agg["fossil_sum"].sum()

1828633.833245781

### add Israeli and non-Israeli holdings

In [26]:
fossil_holdings_by_issuer_agg = pd.concat([il_fossil_holdings_by_issuer_agg, non_il_fossil_holdings_by_issuer_agg])
# clean holding name
fossil_holdings_by_issuer_agg["clean_name"] = fossil_holdings_by_issuer_agg["name"].apply(clean_company)
fossil_holdings_by_issuer_agg["fossil_sum"].sum()

5934869.925977995

In [27]:
# group by issuer_num
fossil_holdings_by_issue_grp = fossil_holdings_by_issuer_agg.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'issuer_num'
], dropna=False)
fossil_holdings_by_issuer_agg = fossil_holdings_by_issue_grp.agg(
    name=pd.NamedAgg(column="clean_name", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()

# group by holding name (clean)
fossil_holdings_by_issue_grp = fossil_holdings_by_issuer_agg.groupby([
    "ParentCorpGroup", "ReportPeriodDate", 'name'
], dropna=False)
fossil_holdings_by_issuer_agg = fossil_holdings_by_issue_grp.agg(
    issuer_num=pd.NamedAgg(column="issuer_num", aggfunc="first"),
    fossil_sum=pd.NamedAgg(column="fossil_sum", aggfunc="sum"),
    quantity_sum=pd.NamedAgg(column="quantity_sum", aggfunc="sum")
).reset_index()


In [28]:
fossil_holdings_by_issuer_agg

,ParentCorpGroup,ReportPeriodDate,name,issuer_num,fossil_sum,quantity_sum
0,אלטשולר,2021-06-30,AESGEN,10134,"161,609.73","54,425,000.00"
1,אלטשולר,2021-06-30,ORO NEGRO DRIL,12824,"6,629.23","23,569,236.01"
2,אלטשולר,2021-06-30,PEMEX,12345,"526,939.93","151,142,000.00"
3,אלטשולר,2021-06-30,WPLAU,13112,"257,804.37","69,625,000.00"
4,אלטשולר,2021-06-30,או פי סי אנרגיה חסום,1682,"294,818.48","9,375,000.00"
5,אלטשולר,2021-06-30,אייסיאל,281,"867,797.50","62,303,664.00"
6,אלטשולר,2021-06-30,אלקו החזקות,694,"65,261.60","336,400.00"
7,אלטשולר,2021-06-30,דלק רכב,829,"715,834.75","15,837,052.00"
8,אלטשולר,2021-06-30,לפידות קפט אג,642,"32,971.36","20,080,000.00"
9,אלטשולר,2021-06-30,תמר פטרוליום,1689,"138,155.27","131,713,683.78"


In [34]:
# divide to quarters (take only first 2)
report_periods = np.sort(fossil_holdings_by_issuer_agg["ReportPeriodDate"].unique())
prev_q = fossil_holdings_by_issuer_agg[fossil_holdings_by_issuer_agg["ReportPeriodDate"] == report_periods[0]]
curr_q = fossil_holdings_by_issuer_agg[fossil_holdings_by_issuer_agg["ReportPeriodDate"] == report_periods[1]]

In [35]:
cols = ["name", "issuer_num", "fossil_sum", "quantity_sum"]
comparison = prev_q[cols].merge(curr_q[cols],
                                on="issuer_num",
                                how="outer",
                                suffixes=["_prev_q", "_curr_q"]
                               )
comparison["name_prev_q"] = comparison["name_prev_q"].fillna(comparison["name_curr_q"])
comparison = comparison.drop("name_curr_q", axis=1).rename({"name_prev_q":"name"}, axis=1)
comparison = comparison.fillna(0).sort_values("name")
comparison["fossil_quantity_diff"] = comparison["quantity_sum_curr_q"] - comparison["quantity_sum_prev_q"]
comparison["fossil_sum_diff"] = comparison["fossil_sum_curr_q"] - comparison["fossil_sum_prev_q"]

In [36]:
company_quarter = company_for_comparison[0] + " - " + quarters_for_comparison[0]
comparison.to_csv("analysis/fossil holdings comparison - " + company_quarter + ".csv", index=False)